<a href="https://colab.research.google.com/github/Guywell/229352-StatisticalLearning/blob/main/229352_Lab04_Naive_Bayes_Grid_and_Random_Search(660510772).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Statistical Learning for Data Science 2 (229352)
#### Instructor: Donlapark Ponnoprat

#### [Course website](https://donlapark.pages.dev/229352/)

## Lab #4

In [16]:
from sklearn.datasets import fetch_20newsgroups
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import classification_report
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV

from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline

from scipy.stats import uniform

In [17]:
train = fetch_20newsgroups(subset='train')
test = fetch_20newsgroups(subset='test')

Xtrain = train.data[:3000]
ytrain = train.target[:3000]
Xtest = test.data[:500]
ytest = test.target[:500]

print("X:", len(Xtest))
print("y:", len(ytest))

X: 500
y: 500


### Naive Bayes [(Documentation)](https://scikit-learn.org/stable/modules/generated/sklearn.naive_bayes.MultinomialNB.html)

In [18]:
from sklearn.naive_bayes import MultinomialNB


nb = MultinomialNB(alpha=0.1)


### Random Search Cross-Validation [(Documentation)](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.RandomizedSearchCV.html)

### Uniform distribution in `Scipy` [(Documentation)](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.uniform.html)

In [19]:
from sklearn.feature_extraction.text import CountVectorizer

pipeline = Pipeline([('count', CountVectorizer(stop_words="english")),
                     ('nb', MultinomialNB())])

parameters = { 'nb__alpha': uniform(loc=0.1, scale=10)}

clf = RandomizedSearchCV(pipeline, parameters, n_iter=10, cv=3)
clf.fit(Xtrain, ytrain)

RandomizedSearchCV(cv=3,
                   estimator=Pipeline(steps=[('count',
                                              CountVectorizer(stop_words='english')),
                                             ('nb', MultinomialNB())]),
                   param_distributions={'nb__alpha': <scipy.stats._distn_infrastructure.rv_continuous_frozen object at 0x7fd844ce5710>})

In [20]:
ypred = clf.predict(Xtest)


print(classification_report(ytest, ypred))

              precision    recall  f1-score   support

           0       0.67      0.67      0.67        21
           1       0.80      0.38      0.52        21
           2       1.00      0.04      0.07        26
           3       0.47      0.79      0.59        34
           4       0.92      0.68      0.78        34
           5       0.57      0.81      0.67        26
           6       1.00      0.73      0.84        22
           7       0.75      0.96      0.84        28
           8       1.00      0.76      0.86        33
           9       0.88      0.84      0.86        25
          10       0.84      1.00      0.92        27
          11       0.54      1.00      0.70        20
          12       0.67      0.42      0.51        24
          13       0.86      0.83      0.84        23
          14       0.81      0.89      0.85        28
          15       0.71      0.93      0.81        29
          16       0.51      0.95      0.67        21
          17       0.82    

/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


#### Exercise

1. For the Naive Bayes model, use grid search 5-fold cross-validation across different values of `alpha` to find the best model.

2. For the best value of `alpha`, compute the `f1_macro` score on the test set.
* What value of `alpha` did you obtain?
* What is the model's `f1_macro` score?

3. Repeat Exercise 1 and 2 for **random search** 5-fold cross validation across different values of `alpha`. Compute the `f1_macro` score on the test set.
* What value of `alpha` did you obtain?
* Did you get a better `f1_macro` score compared to grid search in Exercise 2?

In [23]:
# 1.
pipeline_gs = Pipeline([('count', CountVectorizer(stop_words="english")),
                        ('nb', MultinomialNB())])

# Define the parameter grid for alpha
parameters_gs = {'nb__alpha': [0.001, 0.01, 0.1, 0.5, 1.0, 2.0, 5.0, 10.0]}

# Use GridSearchCV with 5-fold cross-validation (cv=5)
clf_gs = GridSearchCV(pipeline_gs, parameters_gs, cv=5)
clf_gs.fit(Xtrain, ytrain)

print("parameters :", clf_gs.best_params_)
print("(f1_macro) on training data:", clf_gs.best_score_)

parameters : {'nb__alpha': 0.01}
(f1_macro) on training data: 0.8156666666666667


In [24]:
# 2.
from sklearn.metrics import f1_score

# Get the best alpha value from the grid search
best_alpha_gs = clf_gs.best_params_['nb__alpha']

# Make predictions on the test set using the best model
ypred_gs = clf_gs.predict(Xtest)

# Compute the f1_macro score on the test set
f1_macro_gs = f1_score(ytest, ypred_gs, average='macro')

print(f"alpha: {best_alpha_gs}")
print(f"F1_macro score: {f1_macro_gs}")


alpha: 0.01
F1_macro score: 0.7077459540233744


In [26]:
pipeline_rs = Pipeline([('count', CountVectorizer(stop_words="english")),
                        ('nb', MultinomialNB())])

# Define the parameter distribution for alpha (using uniform distribution)
parameters_rs = {'nb__alpha': uniform(loc=0.001, scale=10)} # Adjust the scale and loc as needed

# Use RandomizedSearchCV with 5-fold cross-validation (cv=5) and specify n_iter
clf_rs = RandomizedSearchCV(pipeline_rs, parameters_rs, n_iter=20, cv=5, random_state=42) # Added random_state for reproducibility
clf_rs.fit(Xtrain, ytrain)

print("parameters from random search:", clf_rs.best_params_)

# Get the best alpha value from the random search
best_alpha_rs = clf_rs.best_params_['nb__alpha']

# Make predictions on the test set using the best model from random search
ypred_rs = clf_rs.predict(Xtest)

# Compute the f1_macro score on the test set for the random search result
f1_macro_rs = f1_score(ytest, ypred_rs, average='macro')

print(f"alpha from random search: {best_alpha_rs}")
print(f"F1_macro score (random search): {f1_macro_rs}")

# Compare f1_macro scores
print(f"f1_macro score: {f1_macro_gs}")
print(f"Random search f1_macro score: {f1_macro_rs}")

if f1_macro_rs > f1_macro_gs:
    print("Random search resulted in a better f1_macro score.")
elif f1_macro_rs < f1_macro_gs:
    print("Grid search resulted in a better f1_macro score.")
else:
    print("Grid search and random search resulted in the same f1_macro score.")

parameters from random search: {'nb__alpha': np.float64(0.20684494295802447)}
alpha from random search: 0.20684494295802447
F1_macro score (random search): 0.7289246295687695
f1_macro score: 0.7077459540233744
Random search f1_macro score: 0.7289246295687695
Random search resulted in a better f1_macro score.
